In [1]:
import pandas as pd
import numpy as np
import yfinance
import ta

# Buy Low With Volume Strategy
This strategy taking the newest low in n periods as a trigger with Stochastic RSI oversold, then using the volume as a filter.

### Part 1: Buying Requirements
  1. Stochastic RSI K and D line must be in the oversold area (> n%)
  2. Stochastic RSI K line is below D line
  3. The stock made an n period low during this period of Stochastic RSI oversold
  4. Then check if the volume is above the volume simple moving avreage within a n periods 
  5. If above then Buy at close or at n% above new low

### Part 2: Selling Requirements
  1. Set Trailling stop to n% of high prices
  2. Set sell limit to a static n% gain of buy price or trailling of high prices

## Getting Price Data
Getting the price data of symbol using the yfinance library.

In [2]:
# Loading the 10 years prices data from yfinance
# start: 2010-01-01 end: 2020-12-31
# Symbol: AAPL
symbol = 'AAPL'
ticker = yfinance.Ticker(symbol)
prices = ticker.history(start='2010-01-01', end='2020-12-31', period='1d') 

In [3]:
prices

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-01-04,6.508334,6.540963,6.476315,6.526021,493729600,0.0,0.0
2010-01-05,6.544012,6.574201,6.502845,6.537303,601904800,0.0,0.0
2010-01-06,6.537302,6.563222,6.426609,6.433318,552160000,0.0,0.0
2010-01-07,6.457104,6.464727,6.374770,6.421425,477131200,0.0,0.0
2010-01-08,6.412887,6.464727,6.375075,6.464117,447610800,0.0,0.0
...,...,...,...,...,...,...,...
2020-12-23,130.808961,131.076190,129.443063,129.621231,88223700,0.0,0.0
2020-12-24,129.977561,132.095684,129.759809,130.620911,54930100,0.0,0.0
2020-12-28,132.620283,135.936028,132.145179,135.292679,124486200,0.0,0.0


## Technical Analysis
Using the ta library to do technical analysis.

In [4]:
# Create Stochastic RSI K
stoch_rsi = ta.momentum.StochRSIIndicator(prices.Close, window=20)
stoch_k = stoch_rsi.stochrsi_k()
stoch_d = stoch_rsi.stochrsi_d()
stoch_k

Date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
                ...   
2020-12-23    0.923617
2020-12-24    0.948093
2020-12-28    0.948093
2020-12-29    0.906882
2020-12-30    0.794803
Name: stochrsi_k, Length: 2768, dtype: float64

In [5]:
# Create Simple Moving Average for volumes
volume_sma = ta.trend.sma_indicator(prices.Volume, window=10)
volume_sma

Date
2010-01-04            NaN
2010-01-05            NaN
2010-01-06            NaN
2010-01-07            NaN
2010-01-08            NaN
                 ...     
2020-12-23    116817020.0
2020-12-24    114178810.0
2020-12-28    117933450.0
2020-12-29    122119730.0
2020-12-30    116040570.0
Name: sma_10, Length: 2768, dtype: float64

## Finding the Buy Signal

In [6]:
# Finding lows for 200 period, 100 period, 50 period and 20 period
period_range = [200, 100, 50, 20, 10]
low = prices.Low
# Simple function to find the lowest low in n periods
def get_lowest_low(s, length):
  return s.rolling(length).min()
# Getting a list of lows in different periods
lows = [get_lowest_low(low, _) for _ in period_range]
# Turn the list into a pandas DataFrame with concat
lows = pd.concat(lows, axis=1)
lows.columns = [str(_) + '_day_low' for _ in period_range]
# Check out lows
lows

,200_day_low,100_day_low,50_day_low,20_day_low,10_day_low
Date,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN,NaN
2010-01-06,NaN,NaN,NaN,NaN,NaN
2010-01-07,NaN,NaN,NaN,NaN,NaN
2010-01-08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-23,52.282472,101.870286,106.039946,113.992640,118.921733
2020-12-24,52.282472,101.870286,106.039946,115.031904,119.317644
2020-12-28,52.282472,101.870286,106.039946,115.615871,120.297528


In [7]:
# Concat Low price with Stochastic RSI K and different period lows
columns = ['Low', 'StochRSI_K', 'StochRSI_D'] + lows.columns.tolist()
ndf = pd.concat([prices.Low, stoch_k, stoch_d, lows], axis=1)
# Dropna, with inplace=True so do have to reassign to the same variable
ndf.dropna(inplace=True)
# check the ndf 
ndf

,Low,stochrsi_k,stochrsi_d,200_day_low,100_day_low,50_day_low,20_day_low,10_day_low
Date,,,,,,,,
2010-10-18,9.583960,0.946862,0.866192,5.801484,7.183165,7.183165,8.385850,8.593817
2010-10-19,9.148811,0.846304,0.888288,5.801484,7.183165,7.183165,8.385850,8.698719
2010-10-20,9.357691,0.700217,0.831128,5.801484,7.183165,7.183165,8.385850,8.749031
2010-10-21,9.355558,0.536760,0.694427,5.801484,7.183165,7.183165,8.385850,8.843259
2010-10-22,9.340312,0.491601,0.576193,5.801484,7.183165,7.183165,8.385850,8.919187
...,...,...,...,...,...,...,...,...
2020-12-23,129.443063,0.923617,0.896750,52.282472,101.870286,106.039946,113.992640,118.921733
2020-12-24,129.759809,0.948093,0.918342,52.282472,101.870286,106.039946,115.031904,119.317644
2020-12-28,132.145179,0.948093,0.939934,52.282472,101.870286,106.039946,115.615871,120.297528


This is the logic for getting the buy signal for back test.

In [8]:
# Applying the strategy:
# Check if Stochastic RSI K is lower than 20% (.2), 
# Stochastic RSI D is lower than 20% (.2) and 
# Stochastic RSI K less than Stochastic RSI D
below = .8
low_stoch_rsi = (ndf.stochrsi_k < below) &  (ndf.stochrsi_d < below) & (ndf.stochrsi_k < ndf.stochrsi_d)
# Check if the volume is above the volume sma
good_volume = prices.Volume > volume_sma
# Check if the price is one of the lows
# Using numpy's check to aviod pandas error
is_new_low = np.array(ndf.iloc[:, 2:]) >= np.array(ndf.loc[:, ['Low']])
# Check lowest is lower than more than n periods
is_new_low = is_new_low.sum(1) > 0
print('There are {} new period low in the dataframe of {} period.'.format(
  is_new_low.sum(), len(ndf)))

There are 330 new period low in the dataframe of 2569 period.


In [9]:
# Check for days of possible buys 
# If the Stochastic RSI K is lower then 20% and has an new low
has_hit = pd.concat([low_stoch_rsi, 
                     pd.Series(is_new_low, index=ndf.index)], 
                   axis=1).sum(1) > 1
signals = np.arange(len(has_hit))[has_hit]
print('There are {} signals.'.format(len(signals)))

There are 269 signals.


## Back Test

In [10]:
# Triming prices to the same length as ndf
index = ndf.index
nprices = prices.loc[index, :]
ngood_volume = good_volume.loc[index]

In [17]:
# Backtest loop
is_holding = False
buy_at = 0
stop_limit = 0
stop_limit_pct = 1 - .1
sell_limit = 0
sell_limit_pct = 1 + .5
current_idx = 0
wins = []
num_of_trades = 0
for s in signals:
  # Continue when current index is greater s
  # was still holding
  if current_idx > s:
    continue
  # When there is a signal buy at close
  # getting the next price data and setting the stop limit and sell limit
  data = nprices.iloc[s:, :5]
  for i, (_, row) in enumerate(data.iloc[1:].iterrows()):
    i = i + 1
    if not is_holding:
      if i > 5:
        break
      volume = ngood_volume.iloc[s + i]
      # Buy when good volume
      if volume:
        buy_at = row.Close
        current_high = row.High
        stop_limit = current_high * stop_limit_pct
        sell_limit = buy_at * sell_limit_pct
        is_holding = True
      continue
    # check sell conditions
    is_sell = False
    if row.Open < stop_limit:
      # if open prices is less than stop limit it's a sell
      wp = row.Open / buy_at - 1
      is_sell = True
    elif row.Low < stop_limit:
      # if low prices is less than stop limit, stop limit has been triggered
      wp = stop_limit / buy_at - 1
      is_sell = True
    elif row.High > sell_limit:
      # Sell when sell limit is reached
      wp = sell_limit / buy_at - 1
      is_sell = True
    
    if is_sell:
      # Selling logics
      wins.append(wp)
      current_idx = s + i
      num_of_trades += 1
      is_holding = False
      break

    # Update stop limit when there are new highs
    if row.High > current_high:
      stop_limit = row.High * stop_limit_pct
      sell_limit = row.High * sell_limit_pct
      current_high = row.High  

## Statistics

In [18]:
# Checking the strategy gain
gains = np.array(wins)
strategy_gain = np.prod(gains + 1) - 1
print(f'This Strategy has a gaining rate of : {strategy_gain*100:.2f}%')
print(f'Total trades of {num_of_trades}.') 
win_rate = len(gains[gains > 0]) / len(gains)
avg_win_pct = gains[gains > 0].mean() if win_rate != 0 else 0
biggest_win = gains[gains > 0].max() if win_rate != 0 else 0
avg_lost_pct = gains[gains < 0].mean() if win_rate != 1 else 0
biggest_lost = gains[gains < 0].min() if win_rate != 1 else 0

print(f'''Strategy has a win rate of {win_rate*100:.2f}% 
with an average win percentage of {avg_win_pct*100:.2f}% 
with a biggest win of {biggest_win*100:.2f}%
with an average lost percentage of {avg_lost_pct*100:.2f}% 
with a biggest lost of {biggest_lost*100:.2f}%''')

This Strategy has a gaining rate of : 931.41%
Total trades of 48.
Strategy has a win rate of 52.08% 
with an average win percentage of 18.56% 
with a biggest win of 60.81%
with an average lost percentage of -6.62% 
with a biggest lost of -12.96%


In [13]:
# Seeing how much can be gain through moment
static_gain = nprices.Close[-1] / nprices.Close[0] - 1
print(f'The static gain of {symbol} is :  {static_gain*100:.2f}%') 

The static gain of AAPL is :  1264.87%


In [14]:
# Seeing how much can be gain when sell at the highest price
highest_gain = nprices.High.max() / nprices.Close[0] - 1
print(f'The maximum gain of {symbol} is :  {highest_gain*100:.2f}%') 

The maximum gain of AAPL is :  1316.62%


As you can see this strategy has an advantage to the base strategy, but we can even make it more profitable with some modify to certain parameters. 

The parameters are:
 - below: for setting or adjusting which is the oversold area.
 - stop_limit_pct: for setting the risk level for either a small or larger gain.
 - sell_limit_pct; for a set percentage gain